<a href="https://colab.research.google.com/github/kellemes13/Tool/blob/main/%E6%AD%A1%E8%BF%8E%E4%BD%BF%E7%94%A8_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#import simpy
import numpy as np
#from nand import NAND

CHANNEL_NUM = 8
LUN_NUM = 4
PLANE_NUM = 4
TOTAL_PLANE = CHANNEL_NUM * LUN_NUM * PLANE_NUM
SUPER_PAGE_4K = TOTAL_PLANE * 4

#NAND parameters
SUPER_PAGE_PER_BLOCK = 1536
CHUNK_PER_PAGE = 4
CHUNK_PER_TLC_SUEPR_PAGE = CHANNEL_NUM * LUN_NUM * CHUNK_PER_PAGE*PLANE_NUM * 3
CHUNK_PER_TLC_SUEPR_BLOCK= CHUNK_PER_TLC_SUEPR_PAGE*SUPER_PAGE_PER_BLOCK
MAX_BLK_NUM=300
TLC_BLK_START=0
TLC_GC_TH =3
GC_SEG_UNIT = 384
UNMAP=0xFFFFFFFF


CAPACITY = 2 #TB
#CAPACITY=589824*8*8 #4K_NUM
UTILIZATION = 0.85
real_lba4k_range = int(0.96 * UTILIZATION * CAPACITY * 1024 * 1024 * 256)



L2Etype = np.dtype([('sb_num', np.uint32), ('chunk_idx', np.uint32)])
lpt = np.empty(real_lba4k_range, dtype=L2Etype)
#lpt = np.full(10000, fill_value=(0xFFFFFFFF, 0xFFFFFFFF), dtype=L2Etype)


SuperBlock=[]
TLCFreeblklist=[]
TLCUsedblklist=[]
#SLCFreeblklist=[]
#SLCUsedblklist=[]
CurTLCOpenBlk=None
CurTLCGCBlk=None
CurSLCOpenBlk=None
CurSLCGCBlk=None

Waf=0
GC_segment=0
host_segment=0
TLCGCvictim=None
#SLCGCvictim=None
gc_processing=0



class superblock:
    def __init__(self, env, nd, blknum):
        self.blknum=blknum
        self.vc = 0
        self.ec = 0
        self.cptr = 0
        self.iptr = 0
        self.climit = SUPER_PAGE_PER_BLOCK
        self.p2l = np.empty(SUPER_PAGE_PER_BLOCK * CHUNK_PER_TLC_SUEPR_PAGE, dtype=np.uint32)
        #print('allocate blk',blknum)
        self.env = env
        self.nd = nd

    def write(self, lba4k, nlb4k):
        for i in range(nlb4k):
            self.p2l[i + self.cptr] = lba4k+i
            #lpt[lba4k+i]=((i + self.cptr)|self.blknum<32)
            lpt[lba4k+i]['sb_num']=self.blknum
            lpt[lba4k+i]['chunk_idx']=(i + self.cptr)

        self.cptr += nlb4k
        self.iptr += nlb4k
        self.vc += nlb4k

        # assign write task
        if self.iptr >= SUPER_PAGE_4K:
            self.iptr -= SUPER_PAGE_4K
            #self.env.process(self.nd.write_tlc())

        #print('write blk: blk %d ptr %d LBA4K %d nLBA %d', self.blknum,self.cptr,lba4k,nlb4k)
        if self.cptr==CHUNK_PER_TLC_SUEPR_BLOCK:
            print(f'write blk {self.blknum}, ptr {self.cptr}, LBA4K {lba4k}, NLBA {nlb4k} vc {self.vc}')

    def erase(self):
        self.vc = 0
        self.ec = 0
        self.cptr = 0
        self.p2l = np.empty(SUPER_PAGE_PER_BLOCK * CHUNK_PER_TLC_SUEPR_PAGE, dtype=np.uint32)
        #self.env.process(self.nd.erase())
        print(f'erase blk {self.blknum}')


#no hot/cold separate: 1 new TLC, 1 GC TLC
def FW_write_type1(lba4k,nlb4k):
    global CurTLCOpenBlk
    global TLCGCvictim
    global CurTLCGCBlk
    global host_segment
    global GC_segment
    #GC write

    if len(TLCFreeblklist) <=TLC_GC_TH:
        gc_processing=1
        #calulate waf ,TBD
        if host_segment==0:
            waf=4.6
            GC_segment=int(waf*100*GC_SEG_UNIT)
            host_segment=100*GC_SEG_UNIT
            print(f'[GC]set gc segment {GC_segment} host_segment {host_segment}')
        #process gc write
        while GC_segment!=0:
            #select victim and gc open blk
            if TLCGCvictim is None:
                TLCGCvictim=min(TLCUsedblklist, key=lambda blk: SuperBlock[blk].vc)
                TLCUsedblklist.remove(TLCGCvictim)
                print('[GC] select victim blk',TLCGCvictim,SuperBlock[TLCGCvictim].vc)


            if CurTLCGCBlk is None:
                CurTLCGCBlk=TLCFreeblklist.pop(0)
                SuperBlock[CurTLCGCBlk].erase()
                print('[GC]select TLC GC open blk',CurTLCGCBlk)
            #gc blk is enough
            for idx in range(CHUNK_PER_TLC_SUEPR_BLOCK):
                GCLBA4K=SuperBlock[TLCGCvictim].p2l[idx]
                #valid 4k
                if lpt[GCLBA4K]['sb_num']== TLCGCvictim and lpt[GCLBA4K]['chunk_idx']== idx :
                    SuperBlock[TLCGCvictim].vc-=1
                    SuperBlock[CurTLCGCBlk].write(GCLBA4K,1)
                    if SuperBlock[TLCGCvictim].vc==0:
                        TLCFreeblklist.append(TLCGCvictim)
                        print(f'[GC] blk {TLCGCvictim} add to free! freelen {len(TLCFreeblklist)}' )
                        TLCGCvictim=None


                    if SuperBlock[CurTLCGCBlk].cptr==CHUNK_PER_TLC_SUEPR_BLOCK:
                        TLCUsedblklist.append(CurTLCGCBlk)
                        CurTLCGCBlk=None


                    GC_segment-=1
                    if GC_segment==0:
                        break;

                    if TLCGCvictim is None or CurTLCGCBlk is None:
                        break;

                    #print(f'gc segment_out {GC_segment} host_segment {host_segment}')

        #print(f'[GC] gc segment_out {GC_segment} host_segment {host_segment}')
    else:
        gc_processing=0
        GC_segment=0
        host_segment=0

    #process data write
    if CurTLCOpenBlk is None:
        CurTLCOpenBlk=TLCFreeblklist.pop(0)
        SuperBlock[CurTLCOpenBlk].erase()
        print('[DATA]select TLC  open blk',CurTLCOpenBlk)

    #invalid the prev nand addr
    for i in range(nlb4k):
        oriblk=lpt[lba4k+i]['sb_num']
        if  oriblk !=UNMAP:
            SuperBlock[oriblk].vc-=1
            if  SuperBlock[oriblk].vc==0:
                TLCFreeblklist.append(oriblk)
                print(f'blk {oriblk} add to free!' )

    #blk size is enough
    if SuperBlock[CurTLCOpenBlk].cptr+nlb4k<=CHUNK_PER_TLC_SUEPR_BLOCK:
        SuperBlock[CurTLCOpenBlk].write(lba4k,nlb4k)
        if SuperBlock[CurTLCOpenBlk].cptr==CHUNK_PER_TLC_SUEPR_BLOCK:
            print(f'blk {SuperBlock[CurTLCOpenBlk].blknum} is used!' )
            TLCUsedblklist.append(CurTLCOpenBlk)
            CurTLCOpenBlk =None

    else:
        SuperBlock[CurTLCOpenBlk].write(lba4k,(CHUNK_PER_TLC_SUEPR_BLOCK-SuperBlock[CurTLCOpenBlk].cptr))
        print(f'blk {SuperBlock[CurTLCOpenBlk].blknum} is used!' )
        TLCUsedblklist.append(CurTLCOpenBlk)
        CurTLCOpenBlk=TLCFreeblklist.pop()
        SuperBlock[CurTLCOpenBlk].erase()
        SuperBlock[CurTLCOpenBlk].write(lba4k+(CHUNK_PER_TLC_SUEPR_BLOCK-SuperBlock[CurTLCOpenBlk].cptr),nlb4k-(CHUNK_PER_TLC_SUEPR_BLOCK-SuperBlock[CurTLCOpenBlk].cptr))

    if gc_processing!=0:
        host_segment-=nlb4k
        if host_segment<=0:
            host_segment=0



def InitFW(env):
    # init nand model
    #nd = NAND(env)
    #nd = []
    #for i in range(CHANNEL_NUM * LUN_NUM * PLANE_NUM):
    #    nd[i] = NAND(env)

    #init mapping table
    lpt['sb_num'] = 0xFFFFFFFF
    lpt['chunk_idx'] = 0xFFFFFFFF

    print('real_lba4k_range',real_lba4k_range)
    print('total 4K num',CHANNEL_NUM*LUN_NUM*CHUNK_PER_PAGE*4*1536*3*MAX_BLK_NUM)


    tlcVR = (real_lba4k_range) / (MAX_BLK_NUM * CHUNK_PER_TLC_SUEPR_BLOCK)
    #print(real_lba4k_range,(MAX_BLK_NUM * CHUNK_PER_TLC_SUEPR_BLOCK))
    print('tlcVR',tlcVR)
    tlcVR = tlcVR * tlcVR
    print('tlcVR*tlcVR',tlcVR)
    Waf = 1 + tlcVR/(1-tlcVR)
    print(Waf)



    #init for superblock array
    for blknum in range(MAX_BLK_NUM):
        sb=superblock(None, None, blknum)
        SuperBlock.append(sb)

    #init free TLC blk list
    for blknum in range(TLC_BLK_START,MAX_BLK_NUM):
        #sb=superblock(env, nd, blknum)
        TLCFreeblklist.append(blknum)

    #init free SLC blk
    #for blknum in range(0,TLC_BLK_START):
        #sb=superblock(env, nd, blknum)
    #    SLCFreeblklist.append(blknum)


def main():
    print('InitFW',CHUNK_PER_TLC_SUEPR_BLOCK,real_lba4k_range)


    #env = simpy.Environment()
    InitFW(None)

    #write 7 full blk
    #for i in range(7):
    #    print('write seg',i)
    #    FW_write_type1(i*CHUNK_PER_TLC_SUEPR_BLOCK,CHUNK_PER_TLC_SUEPR_BLOCK)


    print('write cmd start...')
    #W_write_type1(None,589824*7,589824)

    #while 1:
    #    for i in range(4608):
    #        FW_write_type1(589824*7+128*i,128)

    #FW_write_type1(None,16,256)
    #FW_write_type1(None,0,16)
        #print(TLCFreeblklist)
        #print(TLCUsedblklist)
    #env.exit

if __name__ == "__main__":

    main()



#sb = superblock()
#sb.write(0, 100)

#for i in range(110):
#    print(sb.p2l[i])

#sb.write(0, 20)

#for i in range(130):
#    print(sb.p2l[i])

InitFW 2359296 438086664
real_lba4k_range 438086664
total 4K num 707788800
tlcVR 0.6189511108398438
tlcVR*tlcVR 0.3831004776098766
1.6210095221432286
write cmd start...
